# BLAST Results to 3D Protein Structure Visualization

This notebook demonstrates how to:
1. Perform BLAST searches against PDB database
2. Extract PDB IDs from BLAST results
3. Automatically download PDB structures
4. Visualize structures using nglview
5. Create interactive comparisons

## Cell 1: Import Required Libraries

In [11]:
# Import all required libraries
import os
import time
import urllib.request
import pandas as pd
import nglview as nv
from Bio import SeqIO, SearchIO
from Bio.Blast import NCBIWWW
from Bio.PDB import PDBParser
from IPython.display import display
import ipywidgets as widgets

print("All libraries imported successfully!")

All libraries imported successfully!


## Cell 2: Load Your Protein Sequence

In [14]:
# Load your protein sequence
# Option 1: Load from existing single_prot.fasta (if you have it)
try:
    seq_record = SeqIO.read("Sequence_data/single_prot.fasta", "fasta")
    print(f"Loaded sequence from single_prot.fasta")
    print(f"Sequence length: {len(seq_record.seq)}")
    print(f"First 50 amino acids: {seq_record.seq[:50]}")
except FileNotFoundError:
    print("single_prot.fasta not found. Please run the previous notebook first or use Option 2 below.")
    seq_record = None

Loaded sequence from single_prot.fasta
Sequence length: 2701
First 50 amino acids: CTIVFKRVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNC


In [15]:
# Option 2: Create a sample protein sequence for testing
# Uncomment and run this if you don't have single_prot.fasta

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord


sample_protein = " MARIAGINIPDHKHAVIALTSIYGVGKTRSKAILAAAGIAEDVKISELSEGQIDTLRDEVAKFVVEGDLRREISMSIKRLMDLGCYRGLRHRRGLPVRGQRTKTNARTRKGPRKPIKK"

# # Create SeqRecord object
seq_record = SeqRecord(Seq(sample_protein), id="spike_protein", description="SARS-CoV-2 Spike Protein")

print(f"Created sample sequence")
print(f"Sequence length: {len(seq_record.seq)}")
print(f"First 50 amino acids: {seq_record.seq[:50]}")

Created sample sequence
Sequence length: 119
First 50 amino acids:  MARIAGINIPDHKHAVIALTSIYGVGKTRSKAILAAAGIAEDVKISELS


## Cell 3: Perform BLAST Search

In [16]:
%%time
# Perform BLAST search against PDB database
# This may take 2-3 minutes depending on server load

if seq_record is not None:
    print("Starting BLAST search against PDB database...")
    print("This may take a few minutes. Please wait...")
    
    # Perform BLAST search
    result_handle = NCBIWWW.qblast(
        program="blastp",
        database="pdb", 
        sequence=seq_record.seq,
        expect=0.001,  # E-value threshold
        hitlist_size=20  # Number of hits to return
    )
    
    # Parse BLAST results
    blast_results = SearchIO.read(result_handle, "blast-xml")
    
    print(f"BLAST search completed!")
    print(f"Found {len(blast_results)} hits")
    print(f"Query length: {blast_results.seq_len} amino acids")
else:
    print("No sequence loaded. Please run Cell 2 first.")

Starting BLAST search against PDB database...
This may take a few minutes. Please wait...
BLAST search completed!
Found 20 hits
Query length: 118 amino acids
CPU times: total: 3 s
Wall time: 1min 4s


## Cell 4: Extract PDB IDs from BLAST Results

In [17]:
def extract_pdb_ids_from_blast(blast_results, top_n=10):
    """
    Extract PDB IDs and relevant information from BLAST results
    """
    pdb_list = []
    
    for i, hit in enumerate(blast_results[:top_n]):
        # Extract PDB ID from hit ID
        hit_id = hit.id
        
        # Handle different PDB ID formats
        if '|' in hit_id:
            # Format: pdb|7D4F|A or similar
            parts = hit_id.split('|')
            pdb_id = parts[1][:4] if len(parts) > 1 else hit_id[:4]
            chain = parts[2] if len(parts) > 2 else 'A'
        else:
            # Direct PDB ID format
            pdb_id = hit_id[:4]
            chain = 'A'
        
        # Get best HSP (High-scoring Segment Pair)
        best_hsp = hit[0]
        
        pdb_info = {
            'rank': i + 1,
            'pdb_id': pdb_id.upper(),
            'chain': chain,
            'full_id': hit_id,
            'description': hit.description,
            'evalue': best_hsp.evalue,
            'bitscore': best_hsp.bitscore,
            'identity': best_hsp.ident_num,
            'alignment_length': best_hsp.aln_span,
            'query_start': best_hsp.query_start,
            'query_end': best_hsp.query_end,
            'hit_start': best_hsp.hit_start,
            'hit_end': best_hsp.hit_end
        }
        
        pdb_list.append(pdb_info)
    
    return pdb_list

# Extract PDB information
if 'blast_results' in locals():
    pdb_hits = extract_pdb_ids_from_blast(blast_results, top_n=10)
    
    # Display results in a nice table
    df_results = pd.DataFrame(pdb_hits)
    print("Top BLAST Hits:")
    print(df_results[['rank', 'pdb_id', 'evalue', 'bitscore', 'identity', 'description']].to_string(index=False))
else:
    print("No BLAST results found. Please run Cell 3 first.")

Top BLAST Hits:
 rank pdb_id       evalue  bitscore  identity                                                              description
    1   3IYX 7.788200e-82   236.498       118 Chain M, 30S ribosomal protein S13 [Escherichia coli O55:H7 str. CB9615]
    2   7QG8 5.034010e-81   234.572       117               Chain F, 30S ribosomal protein S13 [Escherichia coli K-12]
    3   3J9Z 7.640480e-81   234.187       117                   Chain SM, 30S ribosomal protein S13 [Escherichia coli]
    4   6HRM 3.514520e-80   232.261       116                    Chain r, 30S ribosomal protein S13 [Escherichia coli]
    5   4V4V 1.599070e-79   230.720       115           Chain AM, 30S ribosomal subunit protein S13 [Escherichia coli]
    6   7JIL 1.863190e-79   230.720       114           Chain r, 30S ribosomal protein S13 [Flavobacterium johnsoniae]
    7   2YKR 7.519570e-79   228.794       114         Chain M, 30S RIBOSOMAL PROTEIN S13 [Escherichia coli DH5[alpha]]
    8   4V6Y 8.863060e-79   228.

## Cell 5: Download PDB Structures

In [18]:
def download_pdb_structure(pdb_id, download_dir="Sequence_data"):
    """
    Download PDB structure from RCSB PDB
    """
    # Create directory if it doesn't exist
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    
    pdb_id = pdb_id.upper()
    file_path = os.path.join(download_dir, f"{pdb_id}.pdb")
    
    # Check if file already exists
    if os.path.exists(file_path):
        print(f"✓ PDB file {pdb_id}.pdb already exists")
        return file_path
    
    try:
        # Download from RCSB PDB
        url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
        urllib.request.urlretrieve(url, file_path)
        print(f"✓ Downloaded {pdb_id}.pdb successfully")
        return file_path
    
    except Exception as e:
        print(f"✗ Error downloading {pdb_id}: {e}")
        return None

def batch_download_pdb_structures(pdb_list, max_downloads=5):
    """
    Download multiple PDB structures
    """
    downloaded_files = []
    
    print(f"Downloading top {min(max_downloads, len(pdb_list))} PDB structures...")
    
    for i, pdb_info in enumerate(pdb_list[:max_downloads]):
        pdb_id = pdb_info['pdb_id']
        print(f"\n{i+1}. Downloading {pdb_id}...")
        
        file_path = download_pdb_structure(pdb_id)
        
        if file_path:
            pdb_info['file_path'] = file_path
            downloaded_files.append(pdb_info)
        
        # Small delay to be nice to the server
        time.sleep(0.5)
    
    print(f"\nDownload complete! Successfully downloaded {len(downloaded_files)} structures.")
    return downloaded_files

# Download PDB structures
if 'pdb_hits' in locals():
    downloaded_structures = batch_download_pdb_structures(pdb_hits, max_downloads=5)
else:
    print("No PDB hits found. Please run Cell 4 first.")


1. Downloading 3IYX...
✓ Downloaded 3IYX.pdb successfully

2. Downloading 7QG8...
✗ Error downloading 7QG8: HTTP Error 404: Not Found

3. Downloading 3J9Z...
✗ Error downloading 3J9Z: HTTP Error 404: Not Found

4. Downloading 6HRM...
✗ Error downloading 6HRM: HTTP Error 404: Not Found

5. Downloading 4V4V...
✗ Error downloading 4V4V: HTTP Error 404: Not Found

Download complete! Successfully downloaded 1 structures.


## Cell 6: Basic 3D Visualization

In [19]:
def create_basic_visualization(pdb_info):
    """
    Create basic 3D visualization of a PDB structure
    """
    try:
        # Parse PDB structure
        parser = PDBParser(QUIET=True)  # Suppress warnings
        structure = parser.get_structure(pdb_info['pdb_id'], pdb_info['file_path'])
        
        # Create nglview widget
        view = nv.show_biopython(structure)
        
        # Basic styling
        view.clear_representations()
        view.add_cartoon(color='spectrum')
        
        return view
        
    except Exception as e:
        print(f"Error creating visualization for {pdb_info['pdb_id']}: {e}")
        return None

# Create visualizations for downloaded structures
if 'downloaded_structures' in locals() and len(downloaded_structures) > 0:
    print("Creating 3D visualizations...\n")
    
    for i, pdb_info in enumerate(downloaded_structures[:3]):  # Show first 3
        print(f"=== Structure {i+1}: {pdb_info['pdb_id']} ===")
        print(f"Description: {pdb_info['description'][:80]}...")
        print(f"E-value: {pdb_info['evalue']:.2e}")
        print(f"Bit Score: {pdb_info['bitscore']:.1f}")
        print(f"Identity: {pdb_info['identity']}/{pdb_info['alignment_length']} ({pdb_info['identity']/pdb_info['alignment_length']*100:.1f}%)")
        
        view = create_basic_visualization(pdb_info)
        if view:
            display(view)
        
        print("\n" + "="*80 + "\n")
else:
    print("No downloaded structures found. Please run Cell 5 first.")

Creating 3D visualizations...

=== Structure 1: 3IYX ===
Description: Chain M, 30S ribosomal protein S13 [Escherichia coli O55:H7 str. CB9615]...
E-value: 7.79e-82
Bit Score: 236.5
Identity: 118/118 (100.0%)


NGLWidget()

## Cell 7: Advanced Visualization with Different Styles

In [20]:
def create_styled_visualization(pdb_info, style_type="cartoon"):
    """
    Create styled 3D visualization with different representation options
    """
    try:
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure(pdb_info['pdb_id'], pdb_info['file_path'])
        
        # Create view
        view = nv.show_biopython(structure)
        view.clear_representations()
        
        # Apply different styles
        if style_type == "cartoon":
            view.add_cartoon(color='spectrum')
        elif style_type == "surface":
            view.add_surface(color='hydrophobicity', opacity=0.7)
            view.add_cartoon(color='white', opacity=0.3)
        elif style_type == "ball_stick":
            view.add_ball_and_stick()
        elif style_type == "ribbon":
            view.add_ribbon(color='secondary_structure')
        elif style_type == "spacefill":
            view.add_spacefill(color='element')
        
        # Set camera
        view.camera = 'orthographic'
        
        return view
        
    except Exception as e:
        print(f"Error creating styled visualization: {e}")
        return None

# Create different style visualizations for the best hit
if 'downloaded_structures' in locals() and len(downloaded_structures) > 0:
    best_hit = downloaded_structures[0]  # Best BLAST hit
    
    print(f"Different visualization styles for {best_hit['pdb_id']}:")
    print(f"Description: {best_hit['description'][:60]}...\n")
    
    styles = ["cartoon", "surface", "ribbon"]
    
    for style in styles:
        print(f"--- {style.upper()} Style ---")
        view = create_styled_visualization(best_hit, style)
        if view:
            display(view)
        print()
else:
    print("No structures available for styling. Please run previous cells first.")

Different visualization styles for 3IYX:
Description: Chain M, 30S ribosomal protein S13 [Escherichia coli O55:H7 ...

--- CARTOON Style ---


NGLWidget()


--- SURFACE Style ---


NGLWidget()


--- RIBBON Style ---


NGLWidget()

## Cell 8: Interactive Structure Explorer

In [ ]:
def create_interactive_explorer(downloaded_structures):
    """
    Create interactive widget for exploring different structures and styles
    """
    if not downloaded_structures:
        print("No structures available for interactive exploration.")
        return None
    
    # Create dropdown options
    structure_options = [(f"{s['pdb_id']} (E-val: {s['evalue']:.1e})", s) for s in downloaded_structures]
    
    # Widgets
    structure_dropdown = widgets.Dropdown(
        options=structure_options,
        description='Structure:',
        style={'description_width': 'initial'}
    )
    
    style_dropdown = widgets.Dropdown(
        options=[
            ('Cartoon', 'cartoon'),
            ('Surface', 'surface'), 
            ('Ball & Stick', 'ball_stick'),
            ('Ribbon', 'ribbon'),
            ('Spacefill', 'spacefill')
        ],
        value='cartoon',
        description='Style:'
    )
    
    info_output = widgets.Output()
    viz_output = widgets.Output()
    
    def update_visualization(*args):
        with info_output:
            info_output.clear_output()
            selected_structure = structure_dropdown.value
            
            print(f"PDB ID: {selected_structure['pdb_id']}")
            print(f"Rank: #{selected_structure['rank']}")
            print(f"E-value: {selected_structure['evalue']:.2e}")
            print(f"Bit Score: {selected_structure['bitscore']:.1f}")
            print(f"Identity: {selected_structure['identity']}/{selected_structure['alignment_length']} ({selected_structure['identity']/selected_structure['alignment_length']*100:.1f}%)")
            print(f"Description: {selected_structure['description']}")
        
        with viz_output:
            viz_output.clear_output()
            selected_style = style_dropdown.value
            
            view = create_styled_visualization(selected_structure, selected_style)
            if view:
                display(view)
    
    # Connect events
    structure_dropdown.observe(update_visualization, names='value')
    style_dropdown.observe(update_visualization, names='value')
    
    # Initial display
    update_visualization()
    
    # Create interface
    interface = widgets.VBox([
        widgets.HTML("<h3>Interactive Structure Explorer</h3>"),
        widgets.HBox([structure_dropdown, style_dropdown]),
        info_output,
        viz_output
    ])
    
    return interface

# Create interactive explorer
if 'downloaded_structures' in locals():
    interactive_explorer = create_interactive_explorer(downloaded_structures)
    if interactive_explorer:
        display(interactive_explorer)
else:
    print("No downloaded structures available. Please run previous cells first.")

## Cell 9: Alignment Region Highlighting

In [22]:
def highlight_alignment_region(pdb_info, blast_hit):
    """
    Create visualization with highlighted alignment region
    """
    try:
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure(pdb_info['pdb_id'], pdb_info['file_path'])
        
        # Create view
        view = nv.show_biopython(structure)
        view.clear_representations()
        
        # Get alignment boundaries
        hit_start = pdb_info['hit_start']
        hit_end = pdb_info['hit_end']
        chain_id = pdb_info['chain']
        
        # Add representations
        # Entire structure in gray
        view.add_cartoon(color='gray', opacity=0.3)
        
        # Aligned region in red
        try:
            view.add_cartoon(
                selection=f":{chain_id} and {hit_start}-{hit_end}", 
                color='red'
            )
        except:
            # If specific chain selection fails, highlight entire structure
            view.add_cartoon(color='spectrum')
        
        return view
        
    except Exception as e:
        print(f"Error highlighting alignment: {e}")
        return create_basic_visualization(pdb_info)

# Create alignment highlighting for top hits
if 'downloaded_structures' in locals() and 'blast_results' in locals():
    print("Visualizations with highlighted alignment regions:\n")
    
    for i, pdb_info in enumerate(downloaded_structures[:2]):  # Show first 2
        print(f"=== {pdb_info['pdb_id']} - Alignment Highlighting ===")
        print(f"Aligned region: {pdb_info['hit_start']}-{pdb_info['hit_end']} (chain {pdb_info['chain']})")
        print(f"Query region: {pdb_info['query_start']}-{pdb_info['query_end']}")
        
        # Find corresponding BLAST hit
        blast_hit = blast_results[i]
        
        view = highlight_alignment_region(pdb_info, blast_hit)
        if view:
            display(view)
        
        print("\n" + "="*60 + "\n")
else:
    print("Required data not available. Please run previous cells first.")

Visualizations with highlighted alignment regions:

=== 3IYX - Alignment Highlighting ===
Aligned region: 0-118 (chain M)
Query region: 0-118


NGLWidget()

## Cell 10: Summary and Export Results

In [23]:
# Create summary of results
if 'downloaded_structures' in locals():
    print("=== BLAST to 3D Visualization Summary ===")
    print(f"Query sequence length: {blast_results.seq_len if 'blast_results' in locals() else 'N/A'} amino acids")
    print(f"Total BLAST hits found: {len(blast_results) if 'blast_results' in locals() else 'N/A'}")
    print(f"PDB structures downloaded: {len(downloaded_structures)}")
    print(f"Structures visualized: {min(len(downloaded_structures), 3)}")
    
    print("\n=== Top 5 Hits Summary ===")
    summary_df = pd.DataFrame(downloaded_structures)
    display(summary_df[['rank', 'pdb_id', 'evalue', 'bitscore', 'identity', 'alignment_length']].head())
    
    # Save results to CSV
    summary_df.to_csv('Sequence_data/blast_pdb_results.csv', index=False)
    print("\n✓ Results saved to 'Sequence_data/blast_pdb_results.csv'")
    
    # List downloaded PDB files
    print("\n=== Downloaded PDB Files ===")
    for structure in downloaded_structures:
        if os.path.exists(structure['file_path']):
            file_size = os.path.getsize(structure['file_path']) / 1024  # KB
            print(f"✓ {structure['pdb_id']}.pdb ({file_size:.1f} KB)")
    
else:
    print("No results to summarize. Please run the previous cells first.")

print("\n=== Workflow Complete! ===")
print("You have successfully:")
print("1. ✓ Performed BLAST search against PDB database")
print("2. ✓ Extracted PDB IDs from results")
print("3. ✓ Downloaded PDB structure files")
print("4. ✓ Created 3D visualizations with nglview")
print("5. ✓ Explored different visualization styles")
print("6. ✓ Highlighted alignment regions")

=== BLAST to 3D Visualization Summary ===
Query sequence length: 118 amino acids
Total BLAST hits found: 20
PDB structures downloaded: 1
Structures visualized: 1

=== Top 5 Hits Summary ===


,rank,pdb_id,evalue,bitscore,identity,alignment_length
0,1,3IYX,7.788200e-82,236.498,118,118



✓ Results saved to 'Sequence_data/blast_pdb_results.csv'

=== Downloaded PDB Files ===
✓ 3IYX.pdb (44.9 KB)

=== Workflow Complete! ===
You have successfully:
1. ✓ Performed BLAST search against PDB database
2. ✓ Extracted PDB IDs from results
3. ✓ Downloaded PDB structure files
4. ✓ Created 3D visualizations with nglview
5. ✓ Explored different visualization styles
6. ✓ Highlighted alignment regions
